In [1]:
# %cd '/Users/yashchavan/Desktop/Machine_learning/License_plate_project'

In [2]:
# import pip
# pip.main(["install", "opencv-python",'pyyaml'])
# !pip3 install ipykernel --upgrade
# !python3 -m ipykernel install --user

In [3]:
# !git clone https://github.com/ultralytics/yolov5  # clone repo
# %cd /Users/yashchavan/Desktop/Machine_learning/License_plate_project/yolov5
# # !python -m pip install -qr requirements.txt # install dependencies
# # !pip3 install -q roboflow
# # !pip3 install opencv-python
# # !conda install yaml

# import torch
# import os
# from IPython.display import Image, clear_output  # to display images


In [4]:
# !pip3 install PyYAML

In [5]:
# from roboflow import Roboflow
# rf = Roboflow(model_format="yolov5", notebook="ultralytics")

In [6]:
# os.environ["DATASET_DIRECTORY"] = "/Users/yashchavan/Desktop/Machine_learning/License_plate_project/datasets"
# from roboflow import Roboflow
# rf = Roboflow(api_key="9fHmslgy5tiXsOob0doQ")
# project = rf.workspace().project("license_plate-chwbc")
# dataset = project.version(3).download("yolov5",location="/Users/yashchavan/Desktop/Machine_learning/License_plate_project/datasets")

In [7]:
# !python train.py --img 1280 --batch 10 --epochs 100 --data /Users/yashchavan/Desktop/Machine_learning/License_plate_project/yolov5/data.yaml --hyp hyp.scratch-med.yaml --cache # --weights yolov5m6.pt  # --cfg yolov5m.yaml

In [8]:
# !python detect.py --weights /Users/yashchavan/Desktop/Machine_learning/License_plate_project/yolov5/prev_trained/best.pt --img 1280 --source /Users/yashchavan/Desktop/Machine_learning/License_plate_project/datasets/test/images --save-crop 

In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.nn import CTCLoss
from torch.autograd import Variable
import string
char_list = string.ascii_letters+string.digits

In [10]:
#input shape (height=32,weigth = 128)
class convnet(nn.Module):
    def __init__(self):
        super(convnet, self).__init__()

        self.conv_part = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=64, kernel_size=(3,3), stride=1, padding="same"),#conv1
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)),
            # nn.Dropout(0.25),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride=1, padding="same"),#conv2
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)),
            # nn.Dropout(0.25),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), stride=1, padding="same"),#conv3
            nn.ReLU(),
            # nn.Dropout(0.25),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride=1, padding="same"),#conv4
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,1)),
            # nn.Dropout(0.25),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3,3), stride=1, padding="same"),#conv5
            nn.ReLU(),
            # nn.Dropout(0.25),
            nn.BatchNorm2d(512,512),
            # nn.MaxPool2d(kernel_size=(2,1)),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=1, padding="same"),#conv6
            nn.ReLU(),
            # nn.Dropout(0.25),
            nn.BatchNorm2d(512,512),
            nn.MaxPool2d(kernel_size=(2,1)),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(2,2), stride=1),#conv7
            nn.ReLU(),
            # nn.Dropout(0.25),  
        )

        self.RNN_part1 = nn.Sequential(
            nn.LSTM(input_size =512,batch_first=True,hidden_size=128,bidirectional=True,dropout=0.2,num_layers = 2),
        )
        self.RNN_part2 = nn.Sequential(
            nn.LSTM(input_size =256,batch_first=True,hidden_size=128,bidirectional=True,dropout=0,num_layers = 2),
        )
        
        self.CTC_part = nn.Sequential(
            nn.Linear(256,len(char_list)+1),
            nn.Softmax(dim=2)
        )        
    def forward(self, x):
        out = self.conv_part(x)
        # for layer in self.conv_part:
        #     print(type(layer)," : ",end="")
        #     x = layer(x)x
            # print(x.size())
        # print(out.shape)
        # print(out)
        batch_size, num_channels, height, width = out.shape
        # out = out.permute(3, 0, 1, 2)
        # # print(out.size(1))
        # out = out.reshape((batch_size, out.size(1),-1))
        # # out = out.view(width, batch_size, num_channels * height)
        # # out = out.permute(1, 0, 2)
        # # b, c, h, w = out.shape
        # # assert h == 1, "the height of conv must be 1"
        out = out.squeeze(2)
        # print(out.shape)
        out = out.permute(0,2,1)
        # out = out.permute(2,0,1)

        # print(out.shape)
        # out = out.permute(2, 0, 1)  # [w, b, c]
        # out = out.view(1,b,c*h*w)
        # print(out.shape)
        out, _= self.RNN_part1(out)
        out,_ = self.RNN_part2(out)
        # print(out.shape)
        # x = x
        # for layer in self.conv_part:
        #     x = layer(x)
        #     print(type(layer)," : ",x.size())
        # out, _= self.RNN_part2(out)
        # print(out.shape)
        out = self.CTC_part(out)
        # print(out.shape)
        return out

def ctc_decode(outputs):
    # Get the predicted labels using the CTC decoding algorithm
    _, preds = outputs.max(2)

    # Collapse repeated labels and remove blank symbols
    labels = []
    for pred in preds:
        import itertools
        collapsed = [p for p, _ in itertools.groupby(pred) if p != len(CHARS)]
        labels.append(collapsed)
    
    return labels

In [11]:
# import cv2

# img = cv2.imread("/Users/yashchavan/Desktop/Machine_learning/License_plate_project/text_detect/croped/img_63.jpg")
# img = cv2.resize(img,(32,128))
# img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# img = img.T
# img = img.reshape((-1,1,32,128))
# img = torch.tensor(img).float()
# model = convnet()
# out = model(img)
# out = out.detach().numpy()
# print(out)
# cv2.imshow("image",out)
# cv2.waitKey(0)
# print(out)

In [ ]:
import os
from  scipy.io import loadmat
import pandas as pd
import numpy as np
import cv2

data_img = []
data_label = [] 
data_encoded = []
map_label = {}
data_img_val = []
data_label_val = [] 
data_encoded_val = []
dirc_val_file = "/home/aimlgpu2/Amol/YASH/mnt/ramdisk/max/90kDICT32px/annotation_val.txt"
dirc_lexi = "/home/aimlgpu2/Amol/YASH/mnt/ramdisk/max/90kDICT32px/lexicon.txt"
dirc_train_file = "/home/aimlgpu2/Amol/YASH/mnt/ramdisk/max/90kDICT32px/annotation_train.txt"
dirc_train_imgs = "/content/drive/MyDrive/ML_projects/License_plate_project/IIIT5K/train"
path_weight = "/home/aimlgpu2/Amol/YASH/mnt/ramdisk/max/90kDICT32px/saved_w.txt"

train = pd.read_csv(dirc_train_file)

with open(dirc_lexi) as f:
    for i, text in enumerate(f.readlines()):
        map_label[str(i)] = str(text).rstrip("\n\r")
# print(map_label)
directory = r"/home/aimlgpu2/Amol/YASH/mnt/ramdisk/max/90kDICT32px"
os.chdir(directory)
with open(dirc_train_file) as f:
    count = 0
    for text in (f.readlines()):
        if(count >=7000000):
            break
        temp = text.split(" ")
        # print(temp)
        img = None
        try:
            img = cv2.imread(temp[0],cv2.IMREAD_COLOR)
        except IOError:
            print('Corrupted image for %d' % index)
            continue
        if(not np.any(img)):
            continue
        img = cv2.imread(temp[0],cv2.IMREAD_COLOR)
        img = cv2.resize(img,(128,32))
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        data_img.append(img)
        data_label.append(map_label[temp[-1].rstrip("\n\r")])
        count+=1
    # print(count)
data_img = np.array(data_img)
# print(data_names)
# print(data_img.shape)
# print(map_name_idx)
CHARS = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
CHAR2LABEL = {char: i  for i, char in enumerate(CHARS)}
LABEL2CHAR = {label: char for char, label in CHAR2LABEL.items()}
max_l = 0
for i in range(len(data_label)):
    t = []
    # print(data_label[i])
    for j in range(len(data_label[i])):
        t.append(CHAR2LABEL[data_label[i][j]])
    data_encoded.append(t)
    max_l = max(max_l,len(t))
# print(data_encoded)
# print(t)

for i in range(len(data_encoded)):
    while len(data_encoded[i]) < max_l:
        data_encoded[i].append(len(CHARS))
data_encoded = np.array(data_encoded)
# print(data_encoded.shape)

def convert_towords(output):
    out = []
    for lis in output:
        s = ""
        for i in lis:
            s+=str(LABEL2CHAR[int(i)])
        out.append(s)
    return out

Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file


In [ ]:
# directory = r"/Users/yashchavan/Downloads/datas/mnt/ramdisk/max/90kDICT32px"
# os.chdir(directory)
with open(dirc_val_file) as f:
    count = 0
    for text in (f.readlines()):
        if(count >=700000):
            break
        temp = text.split(" ")
        # print(temp)
        img = None
        try:
            img = cv2.imread(temp[0],cv2.IMREAD_COLOR)
        except IOError:
            print('Corrupted image for %d' % index)
            continue
        if(not np.any(img)):
            continue
            
        img = cv2.imread(temp[0],cv2.IMREAD_COLOR)
        img = cv2.resize(img,(128,32))
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        data_img_val.append(img)
        data_label_val.append(map_label[temp[-1].rstrip("\n\r")])
        count+=1
data_img_val = np.array(data_img_val)
max_l = 0
for i in range(len(data_label_val)):
    t = []
    for j in range(len(data_label_val[i])):
        t.append(CHAR2LABEL[data_label_val[i][j]])
    data_encoded_val.append(t)
    max_l = max(max_l,len(t))

for i in range(len(data_encoded_val)):
    while len(data_encoded_val[i]) < max_l:
        data_encoded_val[i].append(len(CHARS))
data_encoded_val = np.array(data_encoded_val)
print(data_img_val.shape,data_encoded_val.shape)

In [ ]:
batch_size = 300
train_tensor = torch.utils.data.TensorDataset(torch.from_numpy(data_img),torch.from_numpy( data_encoded) )
train_loader = torch.utils.data.DataLoader(train_tensor, batch_size = batch_size,shuffle = True)
val_tensor = torch.utils.data.TensorDataset(torch.from_numpy(data_img_val),torch.from_numpy( data_encoded_val) )
val_loader = torch.utils.data.DataLoader(val_tensor, batch_size = batch_size,shuffle = False)
# from sys import getsizeof

# print(getsizeof(data_img)/ 1024 / 1024,getsizeof(data_encoded)/ 1024 / 1024,getsizeof(data_label)/ 1024 / 1024,)

In [ ]:
epochs = 50
import time
# batch_size = 4
ctc_loss = CTCLoss(blank=len(char_list), reduction='mean', zero_infinity=True)
model = convnet()
model.to("cuda")
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0005)
val_loss = 1e5
for epoch in range(epochs):
    st = time.time()
    total_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
    # for j in range(0,data_img.shape[0],2):
        model.train()
        local_X = data.view(data.shape[0],1,32,128).float()
        local_X = local_X.to("cuda")
        outputs = model(local_X)
        # print(outputs)
        # break
        ll = torch.nn.LogSoftmax(dim=2)
        
#         outputs = ll(outputs)
        outputs = torch.log(outputs)

        targets = target.to("cuda")
        outputs = outputs.transpose(0,1).float()
        output_lengths = torch.full(size=(outputs.size(1),), fill_value=outputs.shape[0], dtype=torch.long).to("cuda")
        target_lengths = torch.full(size=(targets.size(0),), fill_value=targets.shape[1], dtype=torch.long).to("cuda")
        # outputs = outputs.to("cuda")
        loss = ctc_loss(outputs, targets, output_lengths, target_lengths)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
#         print(batch_idx,loss.item())
#         break
    temp_loss = 0
    model.eval()
    for batch_idx, (data, target) in enumerate(val_loader):
    # for j in range(0,data_img.shape[0],2):
        local_X = data.view(data.shape[0],1,32,128).float()
        local_X = local_X.to("cuda")
        outputs = model(local_X)
        # print(outputs)
        # break
#         outputs =  torch.nn.LogSoftmax(outputs,dim=2)
        ll = torch.nn.LogSoftmax(dim=2)
#         outputs = ll(outputs)
        outputs = torch.log(outputs)

        targets = target.to("cuda")
        outputs = outputs.transpose(0,1).float()
        output_lengths = torch.full(size=(outputs.size(1),), fill_value=outputs.shape[0], dtype=torch.long).to("cuda")
        target_lengths = torch.full(size=(targets.size(0),), fill_value=targets.shape[1], dtype=torch.long).to("cuda")
        # outputs = outputs.to("cuda")
        loss = ctc_loss(outputs, targets, output_lengths, target_lengths)
        temp_loss += loss.item()
#         print(batch_idx,loss.item())
#         break
    if(temp_loss<val_loss):
        torch.save(model.state_dict(), path_weight)
    val_loss = temp_loss
    total_loss = total_loss
    print(epoch," : ", "train_loss : ",total_loss, ", val_loss: ",val_loss,time.time()-st)

In [17]:
# model.to("cuda")
# model.load_state_dict(torch.load("/Volumes/NO NAME/ml/w.txt"))
# model.to("cpu")
model.eval()
img = cv2.imread("/home/aimlgpu2/Amol/YASH/mnt/ramdisk/max/90kDICT32px/1/1/117_wrecks_87388.jpg")
img = cv2.resize(img,(32,128))
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img = img.T
# img = img.reshape((-11,32,128))
local_X = torch.tensor([img]).view(1,1,32,128).float().to("cuda")
out = model(local_X)
# out = out.argmax(dim=2)
out = out.detach()
print(out.argmax(2))
pred = ctc_decode(out) 
pred = convert_towords(pred)
print(out)
print(pred)

tensor([[12, 24, 23, 23, 14, 14, 29, 18, 18, 18, 29, 18, 21, 18, 62, 62, 62, 62,
         62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62]], device='cuda:0')
tensor([[[1.6256e-08, 1.7001e-08, 1.6257e-08,  ..., 1.5312e-08,
          1.6589e-08, 2.2160e-05],
         [1.4112e-08, 1.4393e-08, 1.3981e-08,  ..., 1.4065e-08,
          1.3694e-08, 2.4829e-05],
         [2.7451e-08, 2.9460e-08, 2.9233e-08,  ..., 2.9304e-08,
          2.8409e-08, 5.1494e-03],
         ...,
         [2.1379e-10, 3.1600e-10, 2.4112e-10,  ..., 2.1626e-10,
          1.9660e-10, 9.9976e-01],
         [2.1273e-10, 2.7490e-10, 2.2535e-10,  ..., 1.9819e-10,
          1.9237e-10, 1.0000e+00],
         [2.0951e-11, 2.5241e-11, 2.2552e-11,  ..., 1.9200e-11,
          1.8723e-11, 9.9998e-01]]], device='cuda:0')
['conetitili']
